## Setup Cloud Function

Function is set up as an echo for a first test.

In [ ]:
## Requirements.txt
# Function dependencies, for example:
# package>=version
google-cloud-storage
pandas
flask
entsoe-py

## Function Code
def echo2(request):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    
    from flask import jsonify
    
    
    data = {"success": False}
    params = request.get_json()
    
    if "msg" in params:
        data["success"] = True
        data["response"] = str(params["msg"])
                            
    return jsonify(data)
    
#    request_json = request.get_json()
#    if request.args and 'message' in request.args:
#        return request.args.get('message')
#    elif request_json and 'message' in request_json:
#        return request_json['message']
#    else:
#        return f'Hello World!'

In [1]:
import requests

In [2]:
url="https://us-central1-ml-energy-dashboard.cloudfunctions.net/return-load-1"
result = requests.post(url, json={"msg": "Function is working"})

In [3]:
print(result.json())

{'response': 'Function is working', 'success': True}


### Function to return the energy load for a given date range.

In [ ]:
def load_and_persist(request):
    from flask import jsonify
    from entsoe import EntsoePandasClient
    from datetime import datetime, timedelta
    import pandas as pd
    
    def get_date_range(date=None, period=None):

        if date == None:
            date = datetime.today()+timedelta(1)

        if period==None:
            delta = timedelta(14)
        else:
            delta = timedelta(period)

        end = pd.Timestamp(date.strftime('%Y%m%d'), tz='UTC')
        start = pd.Timestamp((date - delta).strftime('%Y%m%d'), tz='UTC')

        return start, end


    def get_entsoe_data(country='ES', date=None, period=None):

        start, end = get_date_range()

        client = EntsoePandasClient(api_key='909addb7-e4ae-4702-acc7-6b4f4fd9667b')

        data = client.query_load(country, start=start, end=end)

        return data

    def make_predictions(df, date=None, period=None):

        #move forward 24 hours. persistance model.
        df = df.shift(24)

        return df

    def load_data():
        df_load = get_entsoe_data()

        df_preds = make_predictions(df_load)

        return df_load, df_preds

    
    data = {"success": False}
    params = request.get_json()
    
    if "get_load" in params and params["get_load"]:
        
        df_load, df_preds = load_data()
        
        data["success"] = True
        data["response"] = dict([("df_load", df_load),
                                 ("df_preds", df_preds)])
                            
    return jsonify(data)
    

In [4]:
#getting error that series object cant be json serialized. how to solve?

In [11]:
from flask import jsonify
from entsoe import EntsoePandasClient
import pandas as pd

client = EntsoePandasClient(api_key='909addb7-e4ae-4702-acc7-6b4f4fd9667b')
data = client.query_load("ES", 
                         start=pd.Timestamp("20200101", tz='UTC'), 
                         end=pd.Timestamp("20200130", tz='UTC'))

In [13]:
data.to_json()

'{"1577836800000":22162.0,"1577840400000":20891.0,"1577844000000":19708.0,"1577847600000":18960.0,"1577851200000":18697.0,"1577854800000":18933.0,"1577858400000":19326.0,"1577862000000":19259.0,"1577865600000":20008.0,"1577869200000":21791.0,"1577872800000":23172.0,"1577876400000":23768.0,"1577880000000":24322.0,"1577883600000":24285.0,"1577887200000":23049.0,"1577890800000":22610.0,"1577894400000":23095.0,"1577898000000":25336.0,"1577901600000":26609.0,"1577905200000":27554.0,"1577908800000":27858.0,"1577912400000":27007.0,"1577916000000":24776.0,"1577919600000":22419.0,"1577923200000":20561.0,"1577926800000":19370.0,"1577930400000":18788.0,"1577934000000":18836.0,"1577937600000":19626.0,"1577941200000":22245.0,"1577944800000":25840.0,"1577948400000":29000.0,"1577952000000":31699.0,"1577955600000":33289.0,"1577959200000":33705.0,"1577962800000":33466.0,"1577966400000":33499.0,"1577970000000":32702.0,"1577973600000":31747.0,"1577977200000":31416.0,"1577980800000":31634.0,"1577984400000

In [24]:
str(data.to_json())

'{"1577836800000":22162.0,"1577840400000":20891.0,"1577844000000":19708.0,"1577847600000":18960.0,"1577851200000":18697.0,"1577854800000":18933.0,"1577858400000":19326.0,"1577862000000":19259.0,"1577865600000":20008.0,"1577869200000":21791.0,"1577872800000":23172.0,"1577876400000":23768.0,"1577880000000":24322.0,"1577883600000":24285.0,"1577887200000":23049.0,"1577890800000":22610.0,"1577894400000":23095.0,"1577898000000":25336.0,"1577901600000":26609.0,"1577905200000":27554.0,"1577908800000":27858.0,"1577912400000":27007.0,"1577916000000":24776.0,"1577919600000":22419.0,"1577923200000":20561.0,"1577926800000":19370.0,"1577930400000":18788.0,"1577934000000":18836.0,"1577937600000":19626.0,"1577941200000":22245.0,"1577944800000":25840.0,"1577948400000":29000.0,"1577952000000":31699.0,"1577955600000":33289.0,"1577959200000":33705.0,"1577962800000":33466.0,"1577966400000":33499.0,"1577970000000":32702.0,"1577973600000":31747.0,"1577977200000":31416.0,"1577980800000":31634.0,"1577984400000

In [37]:
data.index

DatetimeIndex(['2020-01-01 01:00:00+01:00', '2020-01-01 02:00:00+01:00',
               '2020-01-01 03:00:00+01:00', '2020-01-01 04:00:00+01:00',
               '2020-01-01 05:00:00+01:00', '2020-01-01 06:00:00+01:00',
               '2020-01-01 07:00:00+01:00', '2020-01-01 08:00:00+01:00',
               '2020-01-01 09:00:00+01:00', '2020-01-01 10:00:00+01:00',
               ...
               '2020-01-29 15:00:00+01:00', '2020-01-29 16:00:00+01:00',
               '2020-01-29 17:00:00+01:00', '2020-01-29 18:00:00+01:00',
               '2020-01-29 19:00:00+01:00', '2020-01-29 20:00:00+01:00',
               '2020-01-29 21:00:00+01:00', '2020-01-29 22:00:00+01:00',
               '2020-01-29 23:00:00+01:00', '2020-01-30 00:00:00+01:00'],
              dtype='datetime64[ns, Europe/Madrid]', length=696, freq=None)

In [25]:
# Call function from google
url="https://us-central1-ml-energy-dashboard.cloudfunctions.net/return-load"
result = requests.post(url, json={"date": '20200101'})

In [31]:
result.json()['df_load']

'{"1582156800000":24920.0,"1582160400000":23889.0,"1582164000000":23523.0,"1582167600000":23581.0,"1582171200000":24241.0,"1582174800000":27013.0,"1582178400000":31350.0,"1582182000000":33324.0,"1582185600000":34231.0,"1582189200000":33941.0,"1582192800000":33573.0,"1582196400000":33269.0,"1582200000000":33170.0,"1582203600000":32104.0,"1582207200000":31416.0,"1582210800000":31054.0,"1582214400000":30963.0,"1582218000000":31134.0,"1582221600000":33896.0,"1582225200000":34957.0,"1582228800000":34212.0,"1582232400000":31414.0,"1582236000000":28340.0,"1582239600000":26422.0,"1582243200000":24816.0,"1582246800000":23774.0,"1582250400000":23462.0,"1582254000000":23489.0,"1582257600000":24249.0,"1582261200000":26893.0,"1582264800000":31128.0,"1582268400000":33109.0,"1582272000000":34113.0,"1582275600000":33754.0,"1582279200000":33358.0,"1582282800000":33111.0,"1582286400000":32918.0,"1582290000000":31780.0,"1582293600000":30865.0,"1582297200000":30239.0,"1582300800000":29971.0,"1582304400000

In [46]:
df_one = pd.read_json(result.json()['df_load'],typ='series', orient='records')

df_one.values

array([24920, 23889, 23523, 23581, 24241, 27013, 31350, 33324, 34231,
       33941, 33573, 33269, 33170, 32104, 31416, 31054, 30963, 31134,
       33896, 34957, 34212, 31414, 28340, 26422, 24816, 23774, 23462,
       23489, 24249, 26893, 31128, 33109, 34113, 33754, 33358, 33111,
       32918, 31780, 30865, 30239, 29971, 30048, 32479, 33375, 32964,
       30690, 28248, 26677, 24836, 23458, 22945, 22490, 22413, 23108,
       24298, 25705, 28273, 29669, 29599, 29051, 28848, 28136, 26900,
       26037, 25751, 26105, 28739, 29967, 29923, 28604, 26773, 24588,
       22790, 21605, 20949, 20694, 20734, 21028, 21569, 22235, 24433,
       25824, 26196, 25888, 26043, 25772, 24575, 23859, 23642, 24323,
       27273, 29116, 29263, 28381, 26268, 24318, 22872, 21961, 21520,
       21639, 22416, 25544, 29621, 31776, 33025, 33116, 32831, 32582,
       32329, 31347, 30550, 30039, 29807, 29917, 32805, 34049, 33358,
       30570, 27634, 25813, 24184, 23178, 22868, 22816, 23451, 26037,
       29832, 31663,